In [5]:
%run "..\source\required_files\Startup_py3.py"
sys.path.append(r"..")

import source as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

16676


# 0. Initialize Cell_List class

In [6]:
# initialize cell-list parameter dict
param = {'data_folder':r'Y:\20190305_IMR90_CTP05_DNA-FISH',
         'save_folder':r'D:\Pu_Temp\IMR90_WT',
         'experiment_type': 'DNA',
         'num_threads': 16,
         'correction_folder':r'D:\Corrections\Corrections_201906', # which is used to correct bleedthrough/chromatic abbrevation
         };

# create cell_list object
lst = ia.classes.Cell_List(param, _chosen_fovs=[], _exclude_fovs=[], )
# change some global settings according to the microscope I used
lst.shared_parameters['num_empty_frames'] = 0
lst.shared_parameters['single_im_size'] = [50, 2048, 2048]
lst.shared_parameters['corr_Z_shift'] = False

Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 103
- Number of field of views: 85
- Importing csv file: V:\20200113_CTP04_chr21_IMR90_hexanediol\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
-- Hyb H0R0 exists in this data
-- DAPI exists in hyb: H0R0
- Importing csv file: V:\20200113_CTP04_chr21_IMR90_hexanediol\Analysis\Region_Positions.csv
- header: ['region', 'chr', 'start', 'end', 'midpoint']
-- 651 genomic regions loaded!
97 folders are found according to color-usage annotation.


# 1. Segmentation for all (only for the very first time)

## generate cell_segmentations and manually correct

In [ ]:
%matplotlib notebook
%matplotlib notebook

lst._pick_cell_segmentations(_min_shape_ratio=0.02, _shrink_percent=12, 
                             _save=True, _overwrite=False)

In [ ]:
# run this to save updated nucleus status
%matplotlib inline
new_labels, remove_cts, append_cts = lst._update_cell_segmentations(_return_all=True, 
                                                                    _overwrite_segmentation=False, 
                                                                    _append_radius=90)

In [11]:
plt.close('all')

## another option: translate from segmentation generated in RNA

In [ ]:
# generate rotation matrix
experiment_folder = r'Y:\20190302_IMR90_CTP05_intronFISH\Experiment'
pos_file_before = os.path.join(experiment_folder, 'positions_10x_before.txt')    
pos_file_after = os.path.join(experiment_folder, 'positions_10x_after.txt')    
R,t = alignment_tools.align_manual_points(pos_file_before, pos_file_after)

In [ ]:
# apply rotation and translation according to DAPI image alignments between DNA and RNA experiments
old_seg_folder = r'Y:\20190302_IMR90_CTP05_intronFISH\Analysis\segmentation'
old_dapi_folder = r'Y:\20190302_IMR90_CTP05_intronFISH\H0R0'
old_correction_folder = r'I:\Corrections'
new_correction_folder = r'I:\Corrections'
# load previously generated rotation matrix 
experiment_folder = r'Y:\20190302_IMR90_CTP05_intronFISH\Experiment'
rotation_mat = np.load(os.path.join(experiment_folder, 'rotation.npy'))
# translate segmentations!
new_seg_labels = lst._translate_old_segmentations(old_seg_folder, old_dapi_folder, rotation_mat,_force=True, 
                                                  _old_correction_folder=old_correction_folder,_return_all=True,
                                                  )

# 2. Create cell objects

In [ ]:
%matplotlib inline

fov_ids =  np.arange(1,70)

lst._create_cells_fov(fov_ids, _drift_size=650, _coord_sel=[1000,1000], _load_segmentation=False,
                      _load_exist_info=True, _save=False,
                      _exclude_attrs=['segmentation_label'])

# 3. Crop unique images for each cell

In [ ]:
# crop from original images
lst._crop_image_for_cells('unique', _load_in_ram=False,
                          _overwrite_cell_info=True, _force=False, _save=True)

# 4. Pick chromosomes

## automatically generate chromosomes and manually adjust

In [ ]:
# pick
%matplotlib notebook
%matplotlib notebook
lst._get_chromosomes_for_cells('unique',_max_count=90, _overwrite=False)

## update manually picked chromosomes

In [ ]:
# update
reload(visual_tools)
lst._update_chromosomes_for_cells()

# 5. Multi-fitting spots

In [ ]:
# multi-fitting
lst._spot_finding_for_cells('unique', _max_fitting_threads=16, 
                            _fit_window=60,
                            _overwrite=False, _save=True)

# 6. Pick spots to acquire region coordinates

## pick spots by EM

step 1: analyze 50kb regions spots

In [ ]:
# load required attributes if applies
lst._load_cells_from_files('cell_info', _load_attrs=['unique_spots', 'unique_ids', 'chrom_coords'])

In [ ]:
# whether overwrite picked pts
_overwrite_picked_pts = True
# do EM spot pickiung
lst._pick_spots_for_cells('unique', _pick_type='EM', _num_threads=16,
                          _intensity_th=1.0,
                          _local_size=5, _w_ctdist=3, _w_lcdist=1, _w_int=2, _w_nbdist=1,
                          _ignore_ids=False, 
                          _check_th=-3.5, _check_percentile=5,
                          _save_plot=False, _save_to_info=False, 
                          _overwrite=_overwrite_picked_pts)

# 7. Generate population median distance map / contact probability map

In [ ]:
# median distance map
%matplotlib inline
gaussian_filter_size = 0
average_map, all_maps, failure_rate = lst._calculate_population_map('unique', 'EM', _max_loss_prob=0.25,
                                                                    _stat_type='median',_gfilt_size=gaussian_filter_size,
                                                                    _plot_limits=[0,2000], 
                                                                    _return_all_maps=True, _verbose=False)

In [ ]:
# contact probability map
contact_map, _ = lst._calculate_population_map('unique', 'EM', _max_loss_prob=0.25,
                                               _stat_type='contact', _gfilt_size=gaussian_filter_size,
                                               _plot_limits=[0.0025,0.25], _verbose=False)

# 8. save processed data into single file for further analysis

In [ ]:
# save picked_spots
fov_ids, cell_ids, chrom_ids, dna_spots = [], [], [], []
for _cell in lst.cells:
    if hasattr(_cell, 'EM_picked_unique_spots'):
        _sel_spots = getattr(_cell, 'EM_picked_unique_spots')
        for _chrom_id, _spots in enumerate(_sel_spots):
            fov_ids.append(_cell.fov_id)
            cell_ids.append(_cell.cell_id)
            chrom_ids.append(_chrom_id)
            dna_spots.append(_spots)

print(len(fov_ids))

save_dic = {'fov_ids':fov_ids,
            'cell_ids': cell_ids,
            'chrom_ids': chrom_ids,
            'dna_spots': dna_spots}
pickle.dump(save_dic, open(os.path.join(lst.save_folder, 'IMR90_WT_DNA_spots.pkl'), 'wb'))